In [27]:
import numpy as np
import pandas as pd
import os
import urllib.request
import rpy2.robjects as robjects

In [28]:
# mat_cont: matriz quadrada (n x n) que será reagrupada
# aggregate_indices: lista de listas, cada sublista contém os índices das idades a serem agregadas.
  # Exemplo: [[0, 1], [2, 3]] agrupa as faixas etárias 0 e 1 em um único grupo e as faixas 2 e 3 em outro.
# age_structure (opcional): vetor que especifica a estrutura etária (tamanho relativo de cada grupo etário)
  #Por padrão, assume valores iguais (população uniformemente distribuída).
# silent (opcional): valor booleano
  # define se mensagens de aviso sobre discrepâncias em age_structure devem ser exibidas.
  # True (silencioso).

def aggregate_age_matrix(mat_cont, aggregate_indices, age_structure=None, silent=True):
    if age_structure is None:
        age_structure = np.ones(mat_cont.shape[0])

    age_structure = np.array(age_structure, dtype=float)
    #compara número de linhas com colunas
    if mat_cont.shape[0] != mat_cont.shape[1]:
        raise ValueError("mat_cont is not a square matrix")
    #compara número de faixas etárias
    if len(age_structure) < mat_cont.shape[0]:
        raise ValueError("age_structure is smaller than the linear dimension of mat_cont")

    if len(age_structure) > mat_cont.shape[0]:
        if silent == False:
            print("age_structure is bigger than linear size of mat_cont, aggregating the last values in one")
        #agrupa os últimos pesos
        age_structure[mat_cont.shape[0]-1] = sum(age_structure[mat_cont.shape[0]-1:])
    # matriz quadrada com o número de agrupamentos novos
    new_mat_cont = np.zeros((len(aggregate_indices), len(aggregate_indices)))

    for a, indices_i in enumerate(aggregate_indices):
        for b, indices_j in enumerate(aggregate_indices):
            Na = 0
            for i in indices_i:
                for j in indices_j:
                    new_mat_cont[a, b] += age_structure[i] * mat_cont[i, j]
                Na += age_structure[i]
            new_mat_cont[a, b] /= Na

    return new_mat_cont

In [35]:
# Estrutura etária
# Carrega a distribuição etária da população de Manaus em 2020
# age_structure = pd.read_csv('sample_data/Age_distribution_2020_Manaus.csv')

#Link direto para o arquivo CSV no GitHub (raw)
url1 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/Age_distribution_2020_Manaus-130260.csv'
url2 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/Age_distribution_2020_RIODEJANEIRO-330455.csv'
url3 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/Age_distribution_2020_SAOPAULO-355030.csv'
url4 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/BRA_data.csv'

# Carregar o arquivo CSV sem o cabeçalho
age_structure_Manaus = pd.read_csv(url1, delimiter=";")
age_structure_RJ = pd.read_csv(url2, delimiter=";")
age_structure_SP = pd.read_csv(url3, delimiter=";")
mat_cont = pd.read_csv(url4)

# convertendo o DataFrame para uma lista
age_structure_Manaus = age_structure_Manaus.values.flatten()  # Retorna um array 1D (vetor)
age_structure_RJ = age_structure_RJ.values.flatten()          # Retorna um array 1D (vetor)
age_structure_SP = age_structure_SP.values.flatten()          # Retorna um array 1D (vetor)

# Transformar em matriz (NumPy array)
mat_cont = mat_cont.to_numpy()

# Define os índices para agregação
aggregate_indices = [range(4), range(4, 8), range(8, 12), range(12, 16)]

matcontact_Manaus = aggregate_age_matrix(mat_cont, aggregate_indices, age_structure_Manaus)
matcontact_RJ = aggregate_age_matrix(mat_cont, aggregate_indices, age_structure_RJ)
matcontact_SP = aggregate_age_matrix(mat_cont, aggregate_indices, age_structure_SP)

print (matcontact_Manaus)
print (matcontact_RJ)
print (matcontact_SP)

[[10.09341376  3.78360208  2.70759912  0.95224565]
 [ 3.37886069  7.02569801  3.59282647  1.08961128]
 [ 2.81253507  4.91383586  4.69053766  1.28269405]
 [ 1.828981    2.96068755  2.79251301  1.75223559]]
[[10.09315342  3.79031963  2.70756252  0.95035418]
 [ 3.37432519  7.00210521  3.62139505  1.09849406]
 [ 2.74830937  4.87188234  4.65075677  1.29207998]
 [ 1.79804471  2.8240231   2.75148234  1.73661594]]
[[10.10000523  3.78900106  2.70796952  0.95013553]
 [ 3.37393827  6.98996762  3.63232702  1.10205869]
 [ 2.7604727   4.88005716  4.66212996  1.28903444]
 [ 1.80376629  2.84821539  2.75918256  1.73991773]]
